## Importacion de Librerias

In [124]:
from wikiframe import Say, Extractor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from transform import one_hot_transform,normalize
from utils import txt_list, get_x
import warnings
warnings.simplefilter("ignore")

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.linear_model import HuberRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split, cross_val_score

## Variables Principales

In [79]:
#Instanciar el objeto Extractor
extractor = Extractor('data')

#Extraer en df de ./data
df_dict = extractor.extract_from_csv()

#Concatenar train y test para Hacer one hot encoding 
df_all = pd.concat([df_dict['house_train_raw'],df_dict['houses_test_raw']])

#Extraer la lista de features elegidos en ./features.txt
features = txt_list('features.txt')

#Elegimos los features 
y = df_all['SalePrice']
df_all = df_all[features]


## Procesamiento de datos

In [80]:
df_all = one_hot_transform(df_all,features)

In [81]:
#Normalizar los datos
df_all = normalize(df_all)

In [82]:
df_all['SalePrice'] = list(y)

## Train - Test

In [83]:
#Separando los datos como teníamos antes
train = df_all.iloc[:1460,:]
test = df_all.iloc[1460:,:]

In [85]:
X = train[get_x(df_all)]
y = train['SalePrice']

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25)

## Modelo

In [133]:
estimators = {
    #'ElasticNet': ElasticNet(random_state=0),
    #'Lasso': Lasso(alpha=0.2),
    #'Ridge': Ridge(alpha=1),
    'Huber': HuberRegressor(epsilon=1.35,alpha=0.02),
    #'RandomForest': RandomForestRegressor(),
    #'DecisionTree': DecisionTreeRegressor()
}

In [134]:
for name, estimator in estimators.items():
    score = cross_val_score(estimator, X_train, y_train, cv=5, scoring='neg_mean_squared_log_error')
    print(name, np.abs(score).min())

ElasticNet 0.03651677020045757
Lasso 0.03459385021238665
Ridge 0.03457711309539525
Huber 0.03513054602066233


In [101]:
params_est = {'n_estimators': [100,200,300,400,500,600],  
               'max_features': ['auto', 'sqrt'],  
               'max_depth': [10, 15,20,25], 
               'min_samples_split':  [2, 5, 10], 
               'min_samples_leaf': [1, 2, 4], 
               'bootstrap': [True, False]}
    
params=RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=params_est,n_iter=10,cv=5,verbose=5,random_state=2)

In [ ]:
score = cross_val_score(params, X_train, y_train, cv=5, scoring='neg_mean_squared_log_error')

In [123]:
np.abs(score).min()

0.05797540442464048